In [1]:
from modules.data import data_reader
import pandas as pd
import numpy as np
import os

In [2]:
return_evaluation = data_reader.read_data(match=["return"], location="evaluation")
return_prediction = data_reader.read_data(match=["return"], location="prediction")

In [3]:
# path to evaluation directory
project_folder = os.getcwd()
directory = os.path.abspath(os.path.join(project_folder, "results", "evaluation"))

In [4]:
# Initialize an empty DataFrame to store the aggregated results
aggregated_evaluation = pd.DataFrame()

In [5]:
# List all files in the directory
files = [f for f in os.listdir(directory) if f.endswith('.csv')]

In [6]:
# Process each file
for file in files:
    # Extract parameters from the file name
    base_name = os.path.splitext(file)[0]
    parameters = base_name.split('__')
    param_dict = {}
    for param in parameters:
        key, value = param.split('=')
        param_dict[key] = value

    # Read the CSV file into a DataFrame
    file_path = os.path.join(directory, file)
    df = pd.read_csv(file_path)

    # Add parameter columns to the DataFrame
    for key, value in param_dict.items():
        df[key] = value

    # Append the DataFrame to the aggregated DataFrame
    aggregated_evaluation = pd.concat([aggregated_evaluation, df], ignore_index=True)

In [7]:
param_columns = list(param_dict.keys())
csv_columns = list(df.columns)
all_columns = param_columns + [col for col in csv_columns if col not in param_columns]
aggregated_evaluation = aggregated_evaluation[all_columns]

In [8]:
aggregated_evaluation.rename(columns={"Unnamed: 0" : "model"}, inplace=True)

In [9]:
aggregated_evaluation

,P_L,T,FR,T_O_D,FO,C_L_T_S,FT_S_D,S_D,E_D,FT_L,D_L,model,r2,mse,mae,rmse,mda,mape
0,1,AAPL,daily,return,10,128,2022-07-07,2023-07-07,2024-07-07,250,250,arima,-0.274429,0.000233,0.012304,0.015262,0.333333,0.914984
1,1,AAPL,daily,return,10,128,2022-07-07,2023-07-07,2024-07-07,250,250,lag_llama,-0.278130,0.000234,0.012767,0.015284,0.555556,1.041908
2,1,AAPL,daily,return,10,128,2022-07-07,2023-07-07,2024-07-07,250,250,autoregressor,-1.077641,0.000380,0.016222,0.019486,0.555556,1.660689
3,1,AAPL,daily,return,10,128,2022-07-07,2023-07-07,2024-07-07,250,250,ft_lag_llama,-0.076005,0.000197,0.011359,0.014023,0.555556,0.836015
4,1,AAPL,daily,return,10,32,2022-07-07,2023-07-07,2024-07-07,250,250,arima,-0.329972,0.000243,0.013105,0.015591,0.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,1,AAPL,minutely,stock,10,32,2024-07-01,2024-07-08,2024-07-15,1373,1945,ft_lag_llama,-125.731524,9.008694,2.974939,3.001449,0.444444,0.012913
68,1,AAPL,minutely,stock,10,64,2024-07-01,2024-07-08,2024-07-15,1373,1945,arima,-0.362598,0.096860,0.248962,0.311224,0.555556,0.001080
69,1,AAPL,minutely,stock,10,64,2024-07-01,2024-07-08,2024-07-15,1373,1945,lag_llama,-27.453626,2.022622,1.252569,1.422189,0.555556,0.005436
70,1,AAPL,minutely,stock,10,64,2024-07-01,2024-07-08,2024-07-15,1373,1945,autoregressor,-0.245209,0.088515,0.238000,0.297516,0.444444,0.001032
